<h3><b>Author : Sapnil Patel<b></h3>
<h3><b>Date : 22/12/2023<b></h3>

In [1]:
import os
from dotenv import load_dotenv
from langchain.llms import GooglePalm

In [2]:
# import environment variables from .env files
load_dotenv()

True

In [3]:
api_key=os.environ['api_key']

In [4]:

llm = GooglePalm(google_api_key=api_key,temperature=0.2)

c:\Users\SAPNIL\Desktop\Learnings\LLM_SQL\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(llm("Write me an poem of Fafda"))

**O Fafda, Oh Fafda**

O fafda, oh fafda,
You're a crispy, savory treat.
With your chickpea flour base
And your flavorful spices,
You're the perfect snack for any time of day.

I love your light and airy texture,
And your delicious, slightly spicy flavor.
You're the perfect combination of crunchy and chewy,
And I can't get enough of you.

I love to eat you with a cold glass of milk,
Or as a part of a chaat platter.
You're always the star of the show,
And I can't wait to eat you again soon.

O fafda, oh fafda,
You're my favorite snack of all.
You're the perfect combination of taste and texture,
And I can't get enough of you.


In [6]:
# Let's connect vs code with database with pymysql library
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "sapnil_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

In [7]:
# Check if we have successfully connected or not
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [8]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many total nike white tshirts are left?")



> Entering new SQLDatabaseChain chain...
How many total nike white tshirts are left?
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White'
SQLResult: [(Decimal('250'),)]
Answer:250
> Finished chain.


In [9]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('322'),)]
Answer:322
> Finished chain.


In [10]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('18551'),)]
Answer:18551
> Finished chain.


In [11]:
# qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?" )

In [12]:
qns3 = db_chain.run("""SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """)



> Entering new SQLDatabaseChain chain...
SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('27880.100000'),)]
Answer:27880.1
> Finished chain.


In [13]:
qns4 = db_chain.run("How many dollars will I get if I sell all the tshirts of Levi")



> Entering new SQLDatabaseChain chain...
How many dollars will I get if I sell all the tshirts of Levi
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('437'),)]
Answer:437
> Finished chain.


In [14]:
qns4 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('28334'),)]
Answer:28334
> Finished chain.


In [15]:
qns5 = db_chain.run("How many white color Levi's t-shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t-shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(89,), (90,), (20,), (66,)]
Answer:89
> Finished chain.


In [16]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('265'),)]
Answer:265
> Finished chain.


<h4><b>Few Shot Learning<b></h3>

In [17]:
few_shots = [
    {'Question' : "How many total nike white tshirts are left?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM sapnil_tshirts.t_shirts WHERE brand='Nike' and color='White';",
     'SQLResult': "Result of the SQL query",
     'Answer' : "250"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "18551"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post      discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
      (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
      group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
      """,
     'SQLResult': "Result of the SQL query",
     'Answer': "27880.1"} ,
    {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
     'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "28334"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "265"
    }
    
]

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

emb = embeddings.embed_query("How many white color Levi's shirt I have")

In [20]:
emb[:5]

[0.006943074520677328,
 0.048174694180488586,
 0.02245817892253399,
 -0.014070001430809498,
 0.06808178871870041]

In [21]:
# let's generate thr string of al the values of dictionary
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize[0]

"How many total nike white tshirts are left? SELECT sum(stock_quantity) FROM sapnil_tshirts.t_shirts WHERE brand='Nike' and color='White'; Result of the SQL query 250"

In [22]:
from langchain.vectorstores import Chroma

In [23]:
vectorstore = Chroma.from_texts(texts=to_vectorize,embedding=embeddings,metadatas=few_shots)

In [24]:
from langchain.prompts import  SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(vectorstore=vectorstore,k=2)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store"})

[{'Answer': '250',
  'Question': 'How many total nike white tshirts are left?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM sapnil_tshirts.t_shirts WHERE brand='Nike' and color='White';",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '28334',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]